In [1]:
from pymongo import MongoClient
import sys; sys.path.append('../')
import pandas as pd
from IPython.display import *

In [2]:
host_ip = !gcloud compute instances describe datascience-main --zone=us-east1-b | grep natIP | awk -F': ' '{print $2}'

client = MongoClient(host='mongodb://{}'.format(host_ip[0]), port=27017)
db = client.financials
requests_coll = db.requests

In [10]:
dics = list(requests_coll.find({'contents.identifier':'FB'}))

In [4]:
requests_coll.count_documents({})

173641

In [7]:
requests_coll.find_one()

{'_id': ObjectId('5b87062bda499b8498c896d7'),
 'filename': 'SCG_income_statement_QTR_2012-01-01_1.json',
 'contents': {'data': [{'tag': 'operatingrevenue', 'value': 1036000000.0},
   {'tag': 'totalrevenue', 'value': 1036000000.0},
   {'tag': 'operatingcostofrevenue', 'value': 693000000.0},
   {'tag': 'othercostofrevenue', 'value': 48000000.0},
   {'tag': 'totalcostofrevenue', 'value': 741000000.0},
   {'tag': 'totalgrossprofit', 'value': 295000000.0},
   {'tag': 'depreciationexpense', 'value': 87000000.0},
   {'tag': 'totaloperatingexpenses', 'value': 87000000.0},
   {'tag': 'totaloperatingincome', 'value': 208000000.0},
   {'tag': 'totalinterestexpense', 'value': 71000000.0},
   {'tag': 'totalinterestincome', 'value': 1000000.0},
   {'tag': 'otherincome', 'value': 4000000.0},
   {'tag': 'totalotherincome', 'value': -66000000.0},
   {'tag': 'totalpretaxincome', 'value': 142000000.0},
   {'tag': 'incometaxexpense', 'value': 44000000.0},
   {'tag': 'netincomecontinuing', 'value': 9800000

**Items within each statement type**

In [93]:
pipe = [
    {
        '$unwind': '$contents.data'
    },
    {
        '$project': {
            '_id': 0, 'statement': '$contents.statement', 'tag': '$contents.data.tag'
        }
    },
    {
        '$group': {
            '_id': {
                'statement': '$statement',
                'tag': '$tag'
            },
            'total': { '$sum': 1 }
        }
    },
    {
        '$group': {
            '_id': '$_id.statement',
            'tags': {
                '$push': {
                    'tag': '$_id.tag',
                    'total':'$total'
                }
            }
        }
    }
]
statement_tags = list(requests_coll.aggregate(pipe))

In [137]:
statement_tags_df = pd.DataFrame(statement_tags); statement_tags_df
statement_tags_df = statement_tags_df.groupby('_id').tags.apply(lambda x: pd.DataFrame(x.iloc[0])).reset_index(1,1)
statements = statement_tags_df.index.unique()

In [139]:
for statement in statements:
    display(statement_tags_df.loc[statement].sort_values('total', ascending=False).head(15))

,tag,total
_id,,
balance_sheet,totalliabilitiesandequity,32760
balance_sheet,totalassets,32760
balance_sheet,totalliabilities,32744
balance_sheet,totalequityandnoncontrollinginterests,32608
balance_sheet,cashandequivalents,32498
balance_sheet,totalequity,32494
balance_sheet,totalcommonequity,32454
balance_sheet,commonequity,32333
balance_sheet,retainedearnings,31060


,tag,total
_id,,
calculations,normalizednopat,34244
calculations,opextorevenue,34244
calculations,netincomeqoqgrowth,34244
calculations,ebitdagrowth,34244
calculations,evtonopat,34244
calculations,evtoebit,34244
calculations,evtoinvestedcapital,34244
calculations,dividendyield,34244
calculations,debttoequity,34244


,tag,total
_id,,
cash_flow_statement,netcashfromoperatingactivities,33695
cash_flow_statement,netcashfrominvestingactivities,33658
cash_flow_statement,netchangeincash,33597
cash_flow_statement,netcashfromcontinuinginvestingactivities,33581
cash_flow_statement,netcashfromfinancingactivities,33545
cash_flow_statement,netcashfromcontinuingfinancingactivities,33395
cash_flow_statement,netcashfromcontinuingoperatingactivities,33291
cash_flow_statement,netincome,32728
cash_flow_statement,netincomecontinuing,32519


,tag,total
_id,,
income_statement,netincome,34182
income_statement,netincomecontinuing,34179
income_statement,netincometocommon,34175
income_statement,totalpretaxincome,34173
income_statement,totalrevenue,33971
income_statement,weightedavebasicsharesos,33952
income_statement,weightedavedilutedsharesos,33944
income_statement,basiceps,33734
income_statement,dilutedeps,33689


**Sort by Largest Net Income**

In [82]:
pipe = [
    {
        '$unwind': '$contents.data'
    },
    {
        '$match': {'contents.data.tag': 'netincome'}
    },
    {
        '$project': {
            '_id': 0, 'netincome': '$contents.data.value', 'identifier': '$contents.identifier'
        }
    },
    {
        '$group': {
            '_id': '$identifier',
            'avgNetincome': {'$avg': '$netincome'}
        }
    },
    {
        '$sort': {'avgNetincome': -1}
    },
    {
        '$limit' : 10
    }
]

netincome_sums = list(requests_coll.aggregate(pipeline=pipe))

In [83]:
netincome_sums

[{'_id': 'AAPL', 'avgNetincome': 9870060606.060606},
 {'_id': 'XOM', 'avgNetincome': 7214125000.0},
 {'_id': 'JPM', 'avgNetincome': 5310093750.0},
 {'_id': 'WFC', 'avgNetincome': 4955296875.0},
 {'_id': 'MSFT', 'avgNetincome': 4780181818.181818},
 {'_id': 'CVX', 'avgNetincome': 3966218750.0},
 {'_id': 'VZ', 'avgNetincome': 3498359375.0},
 {'_id': 'IBM', 'avgNetincome': 3376515151.5151515},
 {'_id': 'GOOG', 'avgNetincome': 3221500000.0},
 {'_id': 'GOOGL', 'avgNetincome': 3221500000.0}]